In [1]:
import torch
import numpy as np

In [160]:

class Tensor:
    def __init__(self, data, children=(), _op="", label="", grad=None):
        # ensure numpy array
        self.data = np.array(data, dtype=float)
        self.grad = np.zeros_like(self.data) if grad is None else grad
        self.children = children
        self._op = _op
        self.label = label
        self._backward = lambda: None
        self.shape=self.data.shape

    def __repr__(self):
        return f"Tensor(data={self.data}, grad={self.grad}, op={self._op}, label={self.label})"

    # --- elementwise ops ---
    def __add__(self, other):
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data + other.data, (self, other), _op="+")
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out
    
    def __radd__(self, other):
        # Just reverse the order: addition is commutative
        return self + other
    
    
    def __sub__(self, other):
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data - other.data, (self, other), _op="-")
        def _backward():
            self.grad += out.grad
            other.grad += -out.grad
        out._backward = _backward
        return out
    
    def __rsub__(self, other):
        # Just reverse the order: subtraction is commutative
        return self - other
    
    def __mul__(self, other):
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data * other.data, (self, other), _op="*")
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def __rmul__(self, other):
        # Just reverse the order: multiplication is commutative
        return self * other

    def __truediv__(self, other):
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data / other.data, (self, other), _op="/")

        def _backward():
            # dX = 1/y * dZ
            self.grad += (1 / other.data) * out.grad
            # dY = -x / y^2 * dZ
            other.grad += (-self.data / (other.data ** 2)) * out.grad

        out._backward = _backward
        return out
    
    def __rtruediv__(self, other):
        # Just reverse the order: division is commutative
        return self / other


    # --- reductions ---
    def sum(self):
        out = Tensor(self.data.sum(), (self,), _op="sum")
        def _backward():
            self.grad += np.ones_like(self.data) * out.grad
        out._backward = _backward
        return out

    def mean(self):
        out = Tensor(self.data.mean(), (self,), _op="mean")
        def _backward():
            self.grad += np.ones_like(self.data) * out.grad / self.data.size
        out._backward = _backward
        return out

    # --- matrix multiplication ---
    def matmul(self, other):
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data.dot(other.data), (self, other), _op="matmul")
        def _backward():
            self.grad += out.grad.dot(other.data.T)
            other.grad += self.data.T.dot(out.grad)
        out._backward = _backward
        return out
    
    def pow(self,other):
        if not isinstance(other, (int, float)):
            raise Exception("other must be int or float")

        out = Tensor(np.power(self.data,other), (self,), _op="pow")
        def _backward():
            self.grad += (other*self.data**(other-1))*out.grad
        out._backward = _backward
        return out
    
    def sigmoid(self):
        out = Tensor(1/(1+np.exp(-self.data)), (self,), _op="sigmoid")
        def _backward():
            self.grad += out.data*(1-out.data)*out.grad
        out._backward = _backward
        return out
    
    def relu(self):
        out = Tensor(np.maximum(0,self.data), (self,), _op="relu")
        def _backward():
            self.grad += (self.data > 0).astype(float)*out.grad
        out._backward = _backward
        return out
    
    def tanh(self):
        out = Tensor(np.tanh(self.data), (self,), _op="tanh")
        def _backward():
            self.grad += (1-out.data**2)*out.grad
        out._backward = _backward
        return out
    

    def exp(self):
        out  = Tensor(np.exp(self.data), (self,), _op="exp")
        def _backward():
            self.grad += out.data*out.grad
        out._backward = _backward
        return out
    
    def cos(self):
        out  = Tensor(np.cos(self.data), (self,), _op="cos")
        def _backward():
            self.grad += -np.sin(self.data)*out.grad
        out._backward = _backward
        return out
    
    def sin(self):
        out  = Tensor(np.sin(self.data), (self,), _op="sin")
        def _backward():
            self.grad += np.cos(self.data)*out.grad
        out._backward = _backward
        return out
    
    def ln(self):
        out  = Tensor(np.log(self.data), (self,), _op="log")
        def _backward():
            self.grad += 1/self.data*out.grad
        out._backward = _backward
        return out
    def log(self,base):
        if base is not isinstance(base,(int,float)):
            raise Exception("base must be int or float")
        out = Tensor(np.log(self.data) / np.log(base), (self,), _op=f"log_{base}")

        def _backward():
            self.grad += (1 / (self.data * np.log(base))) * out.grad

        out._backward = _backward
        return out


    def backward(self):
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v.children:
                    build_topo(child)
                topo.append(v)

        build_topo(self)
        self.grad = np.ones_like(self.data)

        for node in reversed(topo):
            node._backward()


In [161]:
a= Tensor([[-1,2,3]])
b = Tensor([[1,2,3]])
c = a+b
print(c) #c


Tensor(data=[[0. 4. 6.]], grad=[[0. 0. 0.]], op=+, label=)


In [162]:
a = torch.tensor([[1,2,3]],requires_grad=True,dtype=torch.float32)
b = torch.tensor([[1,2,3]],requires_grad=True,dtype=torch.float32)
c = a*b
d=c+5
e = d.sum()

In [163]:
import torch
import numpy as np

# --- My Tensor example ---
A = Tensor(np.array([[1.0, 2.0], [3.0, 4.0]]))
B = Tensor(np.array([[5.0, 6.0], [7.0, 8.0]]))

# Addition
C = A + B
# Subtraction
D = C - A
# Matrix multiplication
E = D.matmul(B)
# Sum to make scalar for backward
loss = E.sum()
loss.backward()

print("My Tensor grad A:\n", A.grad)
print("My Tensor grad B:\n", B.grad)

# --- PyTorch equivalent ---
A_torch = torch.tensor([[1.0,2.0],[3.0,4.0]], requires_grad=True)
B_torch = torch.tensor([[5.0,6.0],[7.0,8.0]], requires_grad=True)

C_torch = A_torch + B_torch
D_torch = C_torch - A_torch
E_torch = D_torch @ B_torch
loss_torch = E_torch.sum()
loss_torch.backward()

print("PyTorch grad A:\n", A_torch.grad)
print("PyTorch grad B:\n", B_torch.grad)


My Tensor grad A:
 [[0. 0.]
 [0. 0.]]
My Tensor grad B:
 [[23. 27.]
 [25. 29.]]
PyTorch grad A:
 tensor([[0., 0.],
        [0., 0.]])
PyTorch grad B:
 tensor([[23., 27.],
        [25., 29.]])


In [173]:
import torch
import numpy as np

# Helper to compare gradients
def compare_grad(tensor, torch_tensor, tol=1e-6):
    diff = np.abs(tensor.grad - torch_tensor.grad.numpy())
    print("Grad difference:\n", diff)
    assert np.allclose(tensor.grad, torch_tensor.grad.numpy(), atol=tol), "Gradients do not match!"

# -------------------
# 1️⃣ Matrix Addition + exp
print("=== Addition + exp ===")
A = Tensor(np.array([[1.0, 2.0], [3.0, 4.0]]))
B = Tensor(np.array([[5.0, 6.0], [7.0, 8.0]]))

C = (A + B).exp()
loss = C.sum()
loss.backward()
print("My Tensor grad A:\n", A.grad)
print("My Tensor grad B:\n", B.grad)

# PyTorch
A_t = torch.tensor([[1.0,2.0],[3.0,4.0]], requires_grad=True)
B_t = torch.tensor([[5.0,6.0],[7.0,8.0]], requires_grad=True)
C_t = torch.exp(A_t + B_t)
loss_t = C_t.sum()
loss_t.backward()
print("PyTorch grad A:\n", A_t.grad)
print("PyTorch grad B:\n", B_t.grad)
compare_grad(A, A_t)
compare_grad(B, B_t)

# -------------------
# 2️⃣ Matrix Subtraction + sigmoid
print("\n=== Subtraction + sigmoid ===")
A = Tensor(np.array([[1.0, 2.0], [3.0, 4.0]]))
B = Tensor(np.array([[5.0, 6.0], [7.0, 8.0]]))

C = (A - B).sigmoid()
loss = C.sum()
loss.backward()
print("My Tensor grad A:\n", A.grad)
print("My Tensor grad B:\n", B.grad)

# PyTorch
A_t = torch.tensor([[1.0,2.0],[3.0,4.0]], requires_grad=True)
B_t = torch.tensor([[5.0,6.0],[7.0,8.0]], requires_grad=True)
C_t = torch.sigmoid(A_t - B_t)
loss_t = C_t.sum()
loss_t.backward()
print("PyTorch grad A:\n", A_t.grad)
print("PyTorch grad B:\n", B_t.grad)
compare_grad(A, A_t)
compare_grad(B, B_t)

# -------------------
# 3️⃣ Matrix Multiplication + relu
print("\n=== Matmul + relu ===")
A = Tensor(np.array([[1.0, 2.0], [3.0, 4.0]]))
B = Tensor(np.array([[5.0, 6.0], [7.0, 8.0]]))

C = A.matmul(B).relu()
loss = C.sum()
loss.backward()
print("My Tensor grad A:\n", A.grad)
print("My Tensor grad B:\n", B.grad)

# PyTorch
A_t = torch.tensor([[1.0,2.0],[3.0,4.0]], requires_grad=True)
B_t = torch.tensor([[5.0,6.0],[7.0,8.0]], requires_grad=True)
C_t = torch.nn.functional.relu(A_t @ B_t)
loss_t = C_t.sum()
loss_t.backward()
print("PyTorch grad A:\n", A_t.grad)
print("PyTorch grad B:\n", B_t.grad)
compare_grad(A, A_t)
compare_grad(B, B_t)


=== Addition + exp ===
My Tensor grad A:
 [[   403.42879349   2980.95798704]
 [ 22026.46579481 162754.791419  ]]
My Tensor grad B:
 [[   403.42879349   2980.95798704]
 [ 22026.46579481 162754.791419  ]]
PyTorch grad A:
 tensor([[   403.4288,   2980.9580],
        [ 22026.4648, 162754.7969]])
PyTorch grad B:
 tensor([[   403.4288,   2980.9580],
        [ 22026.4648, 162754.7969]])
Grad difference:
 [[8.99749926e-06 2.07707717e-05]
 [9.51056718e-04 5.45599608e-03]]
Grad difference:
 [[8.99749926e-06 2.07707717e-05]
 [9.51056718e-04 5.45599608e-03]]

=== Subtraction + sigmoid ===
My Tensor grad A:
 [[0.01766271 0.01766271]
 [0.01766271 0.01766271]]
My Tensor grad B:
 [[-0.01766271 -0.01766271]
 [-0.01766271 -0.01766271]]
PyTorch grad A:
 tensor([[0.0177, 0.0177],
        [0.0177, 0.0177]])
PyTorch grad B:
 tensor([[-0.0177, -0.0177],
        [-0.0177, -0.0177]])
Grad difference:
 [[3.59709689e-10 3.59709689e-10]
 [3.59709689e-10 3.59709689e-10]]
Grad difference:
 [[3.59709689e-10 3.597096